# Saliva Analysis – BHI 2021

Saliva Analysis for the Paper "Assessing the Influence of the Inner Clock on the Cortisol Awakening Response and Pre-Awakening Movement" (BHI 2021)

## Setup

In [ ]:
import json
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors.v2019 import cmaps

import biopsykit as bp
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.stats import StatsPipeline
from biopsykit.protocols import CAR

from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.plotting import boxplot_saliva_features
from carwatch_analysis.stats import (
    create_unique_night_id, stats_pipeline_saliva_samples, stats_pipeline_saliva_features
)


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
pg.options["round"] = 3

palette = cmaps.faculties

theme_kwargs = {
    "context": "talk",
    "style": "ticks",
    "palette": palette
}
theme_kwargs_scale = {
    "context": "talk",
    "style": "ticks",
    "palette": palette,
    "font_scale": 1.25
}

sns.set_theme(**theme_kwargs)
#plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["pdf.fonttype"] = 42
plt.close("all")

pad = 0.2

export = True
palette

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
plot_path = result_path.joinpath("plots")
stats_path = result_path.joinpath("statistics")

paper_path = Path(json.load(Path("../paper_path.json").open(encoding="utf-8"))["paper_path"])
paper_img_path = paper_path.joinpath("img")

bp.utils.file_handling.mkdirs([result_path, plot_path, stats_path, paper_img_path])

## Data Import

In [ ]:
sample_times = [i*15 for i in range(5)]
car = CAR()

### Raw Cortisol

In [ ]:
cort_samples = load_long_format_csv(export_path.joinpath("cortisol_samples_cleaned.csv")).reset_index("cortisol")
cort_samples = create_unique_night_id(cort_samples)
cort_samples.head()

### Cortsol Features

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_cleaned.csv"))
cort_features.head()

In [ ]:
car.add_saliva_data(cort_samples, saliva_type="cortisol", sample_times=sample_times)

## Plots and Statistics

In [ ]:
condition_order = ["Spontaneous", "Known Alarm", "Unknown Alarm"]

### Subjects after Cleaning

In [ ]:
subjects = cort_samples.index.get_level_values("subject").unique()
len(subjects)

### Class Distribution

In [ ]:
describe_groups_df(cort_samples, "condition", order=condition_order)

### Raw Cortisol

#### Statistics

In [ ]:
variable = "condition"

pipeline = stats_pipeline_saliva_samples(cort_samples, variable=variable)
pipeline.display_results(posthoc=False)
if export:
    pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_raw_{variable}.xlsx"))

#### Plots

In [ ]:
hue = "condition"

sns.set_theme(**theme_kwargs_scale)
fig, ax = plt.subplots(figsize=(10, 4.5))
car.car_saliva_plot(saliva_type="cortisol", hue=hue, hue_order=condition_order, ax=ax, legend_fontsize="small")

fig.tight_layout(pad=pad)
if export:
    [fig.savefig(p.joinpath(f"img_car_raw_{hue}.pdf"), transparent=True) for p in [plot_path, paper_img_path]]

#### Plots for Presentation

In [ ]:
hue = "condition"

sns.set_theme(**theme_kwargs_scale)
fig, ax = plt.subplots(figsize=(10, 7))
car.car_saliva_plot(saliva_type="cortisol", hue=hue, hue_order=condition_order, ax=ax, legend_fontsize="small")

fig.tight_layout(pad=pad)
if export:
    fig.savefig(plot_path.joinpath("img_car_raw_condition_presentation.pdf"), transparent=True)

### Cortisol Features

#### Statistics

In [ ]:
variable = "condition"

pipeline = stats_pipeline_saliva_features(cort_features, variable=variable, equal_var=True)
pipeline.display_results(posthoc=True, sig_only="posthoc", grouped=False)
if export:
    pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_features_{variable}.xlsx"))

#### Plots

In [ ]:
features = {
    "auc": ["auc_i", "auc_g"],
    "slope": "slopeS0S3",
    "max_inc": "max_inc"
}
hue = "condition"

box_pairs, pvals = pipeline.sig_brackets(
    "posthoc", 
    stats_effect_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    subplots=True, 
    features=features
)

sns.set_theme(**theme_kwargs_scale)
fig, axs = plt.subplots(figsize=(12,5), ncols=len(features))
car.saliva_multi_feature_boxplot(
    data=cort_features,
    saliva_type="cortisol", 
    hue=hue, 
    features=features, 
    hue_order=condition_order, 
    stats_kwargs={
        "box_pairs": box_pairs, 
        "pvalues": pvals,
        "text_offset": -8
    },
    ax=axs, 
    legend_fontsize="small", 
    legend_orientation="horizontal", 
    legend_loc="upper center",
    width=0.9,
    saturation=1.0
)
fig.tight_layout(rect=(0, 0, 1.0, 0.88), pad=pad)

if export:
    [fig.savefig(p.joinpath(f"img_boxplots_car_{hue}.pdf"), transparent=True) for p in [plot_path, paper_img_path]]

#### Plots for Presentation

In [ ]:
features = {
    "auc": ["auc_i", "auc_g"],
    "slope": "slopeS0S3",
    "max_inc": "max_inc"
}
hue = "condition"

box_pairs, pvals = pipeline.sig_brackets(
    "posthoc", 
    stats_effect_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    subplots=True, 
    features=features
)

sns.set_theme(**theme_kwargs_scale)
fig, axs = plt.subplots(figsize=(11,7), ncols=len(features))
car.saliva_multi_feature_boxplot(
    data=cort_features,
    saliva_type="cortisol", 
    hue=hue, 
    features=features, 
    hue_order=condition_order, 
    stats_kwargs={
        "box_pairs": box_pairs, 
        "pvalues": pvals,
        "text_offset": -8
    },
    ax=axs, 
    legend_fontsize="small", 
    legend_orientation="horizontal", 
    legend_loc="upper center",
    width=0.9,
    saturation=1.0
)
fig.tight_layout(rect=(0, 0, 1.0, 0.85), pad=pad)

if export:
    fig.savefig(plot_path.joinpath(f"img_boxplots_car_{hue}_presentation.pdf"), transparent=True)